In [1]:
import os
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 11:58:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .option("header", "true") \
    .parquet('data/price_n_volume/2021/AAPL.parquet')\
    .drop('__index_level_0__')

In [4]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- adjusted_close: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- dividend_amount: double (nullable = true)
 |-- split_coefficient: double (nullable = true)
 |-- close_percent_change: double (nullable = true)
 |-- company: string (nullable = true)



In [5]:
from pyspark.sql.functions import col

df.select(\
          col("company")
    ).groupBy('company').count().show()

+-------+-----+
|company|count|
+-------+-----+
|   AAPL|  252|
+-------+-----+



In [6]:
# source: https://stackoverflow.com/questions/71038595/pyspark-cumulative-sum-within-partition-for-moving-last-2-n-rows
from pyspark.sql import functions as F, Window

w = Window.partitionBy('company').orderBy('date').rowsBetween(Window.currentRow, 2)

df = df.withColumn(
    'close_percent_change_new', 
    F.avg('close_percent_change').over(w))

In [7]:
df.show()

+-------------------+------+--------+-------+------+----------------+---------+---------------+-----------------+--------------------+-------+------------------------+
|               date|  open|    high|    low| close|  adjusted_close|   volume|dividend_amount|split_coefficient|close_percent_change|company|close_percent_change_new|
+-------------------+------+--------+-------+------+----------------+---------+---------------+-----------------+--------------------+-------+------------------------+
|2021-01-04 00:00:00|133.52|133.6116| 126.76|129.41|127.519519957544|143301887|            0.0|              1.0|             -2.4719|   AAPL|                 -1.5339|
|2021-01-05 00:00:00|128.89|  131.74| 128.43|131.01|129.096146431016| 97664898|            0.0|              1.0|              1.2364|   AAPL|     0.42749999999999994|
|2021-01-06 00:00:00|127.72|131.0499|126.382| 126.6|124.750569713508|155087970|            0.0|              1.0|             -3.3662|   AAPL|     0.30306666666

In [8]:
df.toPandas().head()

,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,close_percent_change,company,close_percent_change_new
0,2021-01-04,133.52,133.6116,126.760,129.41,127.519520,143301887,0.0,1.0,-2.4719,AAPL,-1.533900
1,2021-01-05,128.89,131.7400,128.430,131.01,129.096146,97664898,0.0,1.0,1.2364,AAPL,0.427500
2,2021-01-06,127.72,131.0499,126.382,126.60,124.750570,155087970,0.0,1.0,-3.3662,AAPL,0.303067
3,2021-01-07,128.36,131.6300,127.860,130.92,129.007461,109578157,0.0,1.0,3.4123,AAPL,0.650167
4,2021-01-08,132.43,132.6300,130.230,132.05,130.120954,105158245,0.0,1.0,0.8631,AAPL,-0.533800


In [69]:
-3.3662 + 1.2364 -2.4719 

-4.601700000000001

In [9]:
# source: https://stackoverflow.com/questions/71038595/pyspark-cumulative-sum-within-partition-for-moving-last-2-n-rows
from pyspark.sql import functions as F, Window


def window_all():
    path = 'data/price_n_volume/2021/*.parquet'
    df = spark.read \
        .option("header", "true") \
        .parquet(path)\
        .drop('__index_level_0__')
    w = Window.partitionBy('company').orderBy('date').rowsBetween(Window.currentRow, 2)

    df = df.withColumn(
        'close_percent_change_new', 
        F.avg('close_percent_change').over(w))
    
    return df


df_all = window_all()


In [10]:
df_all.select(\
          col("company")
    ).groupBy('company').count().show()

+-------+-----+
|company|count|
+-------+-----+
|   AAPL|  252|
|   AMGN|  252|
|   AMZN|  252|
|    ADI|  252|
|    IBM|  252|
|    AMD|  252|
|   ABNB|  252|
|    AEP|  252|
|   ADBE|  252|
|   ANSS|  252|
|   ALGN|  252|
+-------+-----+



In [11]:
df_all.createOrReplaceTempView("all_stocks")
df_appl = spark.sql("SELECT * FROM all_stocks WHERE company = 'AAPL'")

In [12]:
# Test to check if df does not take into consideration other companies average 

False in (df_appl.toPandas().close_percent_change_new == df.toPandas().close_percent_change_new)

False

In [43]:
from pyspark.sql.functions import udf

def udf_if_outlier(val, avg, stddev):
    try:
        if ( val > (avg + stddev)) or ( val < (avg - stddev)):
            return True
    except Exception as e:
        # print(f'e = {e}\n val = {val}, avg = {avg}, stddev = {stddev}')
        return False
    return False

def udf_if_outlier_high_only(val, avg, stddev):
    try:
        if ( val > (avg + stddev)):
            return True
    except Exception as e:
        return False
    return False


def window_create(df, on_column ,number_of_days, both=1):
    w = Window.partitionBy('company').orderBy('date').rowsBetween(Window.currentRow+1, number_of_days)
    
    avg_column = f'avg_{on_column}_{number_of_days}'
    stddev_column = f'stddev_{on_column}_{number_of_days}'
    outlier_column = f'if_outlier_{on_column}_{number_of_days}'

    df = df.withColumn(
        avg_column, 
        F.avg(f'{on_column}').over(w))
    
    df = df.withColumn(
        stddev_column, 
        F.stddev(f'{on_column}').over(w))
    
    if both:
        udfValueToOutlier = udf(udf_if_outlier, types.BooleanType())
    else:
        udfValueToOutlier = udf(udf_if_outlier_high_only, types.BooleanType())
     
    # here does not work
    
    df = df.withColumn(
            outlier_column, 
            udfValueToOutlier(df[f'{on_column}'], df[f'{avg_column}'], df[f'{stddev_column}']))
    
    df.select(\
          col("company"), col(outlier_column)
    ).groupBy('company', outlier_column).count().orderBy(col("company")).show()
    
    return df


path = 'data/price_n_volume/202*/*.parquet'
df = spark.read \
        .option("header", "true") \
        .parquet(path)\
        .drop('__index_level_0__')

df = window_create(df, 'close_percent_change', 20)
# df.toPandas().head()

+-------+----------------------------------+-----+
|company|if_outlier_close_percent_change_20|count|
+-------+----------------------------------+-----+
|   AAPL|                             false|  558|
|   AAPL|                              true|  264|
|   ABNB|                              true|  193|
|   ABNB|                             false|  387|
|   ADBE|                             false|  553|
|   ADBE|                              true|  265|
|    ADI|                              true|  263|
|    ADI|                             false|  559|
|    AEP|                              true|  263|
|    AEP|                             false|  559|
|   ALGN|                              true|  244|
|   ALGN|                             false|  574|
|    AMD|                              true|  250|
|    AMD|                             false|  568|
|   AMGN|                              true|  246|
|   AMGN|                             false|  576|
|   AMZN|                      

In [45]:
days = 50
df = window_create(df, 'close_percent_change', days)

+-------+----------------------------------+-----+
|company|if_outlier_close_percent_change_50|count|
+-------+----------------------------------+-----+
|   AAPL|                             false|  559|
|   AAPL|                              true|  263|
|   ABNB|                              true|  177|
|   ABNB|                             false|  403|
|   ADBE|                             false|  584|
|   ADBE|                              true|  234|
|    ADI|                              true|  261|
|    ADI|                             false|  561|
|    AEP|                              true|  260|
|    AEP|                             false|  562|
|   ALGN|                              true|  207|
|   ALGN|                             false|  611|
|    AMD|                              true|  250|
|    AMD|                             false|  568|
|   AMGN|                              true|  228|
|   AMGN|                             false|  594|
|   AMZN|                      

In [46]:
days = 50
df = window_create(df, 'volume', days, both=0)

+-------+--------------------+-----+
|company|if_outlier_volume_50|count|
+-------+--------------------+-----+
|   AAPL|               false|  661|
|   AAPL|                true|  161|
|   ABNB|                true|   83|
|   ABNB|               false|  497|
|   ADBE|               false|  683|
|   ADBE|                true|  135|
|    ADI|                true|  132|
|    ADI|               false|  690|
|    AEP|                true|  104|
|    AEP|               false|  718|
|   ALGN|                true|  113|
|   ALGN|               false|  705|
|    AMD|                true|  147|
|    AMD|               false|  671|
|   AMGN|                true|   93|
|   AMGN|               false|  729|
|   AMZN|               false|  693|
|   AMZN|                true|  125|
|   ANSS|                true|  132|
|   ANSS|               false|  690|
+-------+--------------------+-----+
only showing top 20 rows



In [47]:
days = 100
df = window_create(df, 'volume', days, both=0)

+-------+---------------------+-----+
|company|if_outlier_volume_100|count|
+-------+---------------------+-----+
|   AAPL|                false|  660|
|   AAPL|                 true|  162|
|   ABNB|                 true|   69|
|   ABNB|                false|  511|
|   ADBE|                false|  696|
|   ADBE|                 true|  122|
|    ADI|                 true|  116|
|    ADI|                false|  706|
|    AEP|                 true|   90|
|    AEP|                false|  732|
|   ALGN|                 true|  102|
|   ALGN|                false|  716|
|    AMD|                 true|  134|
|    AMD|                false|  684|
|   AMGN|                 true|   85|
|   AMGN|                false|  737|
|   AMZN|                false|  703|
|   AMZN|                 true|  115|
|   ANSS|                 true|  128|
|   ANSS|                false|  694|
+-------+---------------------+-----+
only showing top 20 rows



In [48]:
days = 200
df = window_create(df, 'volume', days, both=0)

+-------+---------------------+-----+
|company|if_outlier_volume_200|count|
+-------+---------------------+-----+
|   AAPL|                false|  667|
|   AAPL|                 true|  155|
|   ABNB|                 true|   70|
|   ABNB|                false|  510|
|   ADBE|                false|  707|
|   ADBE|                 true|  111|
|    ADI|                 true|  116|
|    ADI|                false|  706|
|    AEP|                 true|   94|
|    AEP|                false|  728|
|   ALGN|                 true|  104|
|   ALGN|                false|  714|
|    AMD|                 true|  150|
|    AMD|                false|  668|
|   AMGN|                 true|   87|
|   AMGN|                false|  735|
|   AMZN|                false|  696|
|   AMZN|                 true|  122|
|   ANSS|                 true|  134|
|   ANSS|                false|  688|
+-------+---------------------+-----+
only showing top 20 rows



![Outlier](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Standard_deviation_diagram.svg/1920px-Standard_deviation_diagram.svg.png)

In [35]:
def udf_if_outlier(val, avg, stddev):
    if ( val > (avg + stddev)) or ( val < (avg - stddev)):
        return 1
    return 0

print(udf_if_outlier(0.8, 1, 0.1),udf_if_outlier(1.2, 1, 0.1) )

1 1


1